In [ ]:
# from langchain_ollama import ChatOllama

# mistral = ChatOllama(
#     model="mistral",
#     temperature=0,
# )

In [ ]:
# from langchain_ollama import OllamaEmbeddings

# embeddings = OllamaEmbeddings(
#     model="mistral",
# )

###  Using Groqcloud API

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("GROQ_API_KEY")

In [ ]:
from langchain_groq import ChatGroq

groqLLM = ChatGroq(
    model="deepseek-r1-distill-qwen-32b", 
    api_key=api_key,
    temperature=0.7, 
    max_tokens=512    # Limit response length
)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import tqdm as notebook_tqdm

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [ ]:
result = embeddings.embed_query("hello")

In [ ]:
len(result)

In [ ]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
import time

index_name = "personal" 

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
# index.delete(delete_all=True)

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader

def custom_loader(file_path: str):
    if file_path.endswith(".pdf"):
        return PyPDFLoader(file_path)
    elif file_path.endswith(".txt"):
        return TextLoader(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_path}")

loader = DirectoryLoader("personal", glob="**/*", show_progress=True, loader_cls=custom_loader)
docs = loader.load()

In [ ]:
# vector_store.add_documents(docs)

In [ ]:
groq_retreiver = vector_store.as_retriever(search_kwargs={'k':2})

In [ ]:
from langchain_core.tools import tool

@tool
def marks_reader(self, file_path:str) -> str:
    """Read content from a text file."""
    try:
        if not os.path.exists(file_path):
            return f"Error: File not found at path {file_path}"
        
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        return content
    except Exception as e:
        return f"Error reading file: {str(e)}"
    
tools = [marks_reader]
groqLLM_with_tools = groqLLM.bind_tools(tools)

In [ ]:
from langchain.prompts import PromptTemplate

from langchain_core.prompts import ChatPromptTemplate

template = """
You are a digital copy of this user that can provide information based on his documents using both vector search from a database and other tools.

### Task:
You will be provided with a user query. Your goal is to respond to the query by doing the following:
1. Retrieve relevant information from the vector database (Pinecone) that best matches the query, provided in the 'Relevant Documents' section.
2. If additional context or information is required beyond the retrieved documents, use the appropriate tool (e.g., marks_reader for reading marks from files).
3. Combine the information from the database and tools to provide a well-structured and complete response.

### User Query:
{user_query}

### Relevant Documents (from Pinecone vector database):
{retrieved_documents}

### Instructions:
- If the retrieved documents are sufficient, respond directly with a concise, clear, and comprehensive answer.
- If the documents are insufficient and a tool is needed (e.g., to fetch marks from 'marks.txt'), invoke the appropriate tool using the tool-calling protocol. Do not output raw JSON; instead, use the tool and include its result in your response.
- If you use a tool, briefly explain in your response how it was used (e.g., "I used the marks_reader tool to fetch your marks from marks.txt").

### Answer:
Provide your response here, integrating information from the retrieved documents and any tool results.
"""

prompt = ChatPromptTemplate.from_template(template)

# prompt = PromptTemplate(
#     input_variables=["user_query", "retrieved_documents"],
#     template=template,
# )

In [ ]:
chain = prompt | groqLLM_with_tools

In [ ]:
chain.invoke({"user_query": "Hello how are you",
              "retrieved_documents": "there are no documents"})

In [ ]:
# user_query = input("Your query :: \n")
# retrieved_documents = mistral_retriever.invoke(user_query)

# chain = prompt | mistral_with_tools

# response = chain.invoke({"user_query": user_query, "retrieved_documents": retrieved_documents})
# print(response.content)

In [ ]:
# print(response.content)

### Let's add memory to the chatbot using **langgraph**

In [ ]:
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import START, END, StateGraph
import operator
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

class GraphState(TypedDict):
    user_query : str
    retrieved_documents : Sequence[str]
    messages: Annotated[list, add_messages]


def input_node(state: GraphState) -> GraphState:
    if not state.get("messages"):
        state["messages"] = [HumanMessage(content=state["user_query"])]
    return state

def retrieval_node(state: GraphState)->GraphState:
    retrieved_documents = groq_retreiver.invoke(state['user_query'])
    state['retrieved_documents'] = retrieved_documents
    return state

def processing_node(state: GraphState)->GraphState:
    chain = prompt | groqLLM_with_tools
    response = chain.invoke(
        {
            "messages": state["messages"],
            "user_query": state['user_query'],
            "retrieved_documents": state['retrieved_documents']
        }
    )
    state["messages"] = state["messages"] + [response]
    return state

tool_node = ToolNode(tools=tools)

workflow = StateGraph(GraphState)

workflow.add_node("input", input_node)
workflow.add_node("retrieval", retrieval_node)
workflow.add_node("processing", processing_node)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "input")
workflow.add_edge("input","retrieval")
workflow.add_edge("retrieval","processing")

workflow.add_conditional_edges("processing", tools_condition)  # if tool call is there, it is called
workflow.add_edge("tools", "processing")  # if toolcall is made then tool execution should occur, if occur, this is ..

workflow.add_edge("processing",END)

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "1"}} 

print("Start chatting! Type 'exit' to quit.")
while True:
    user_query = input("You: ")
    if user_query.lower() == "exit":
        print("Goodbye!")
        break
    
    initial_state = {
        "user_query": user_query,
        "retrieved_documents": [],
        "messages": app.get_state(config).values.get("messages", [])  # Load previous messages
    }
    
    result = app.invoke(initial_state, config=config)
    print("Assistant:", result["messages"][-1].content)